In [ ]:
import os

import pdir
import pandas as pd

In [ ]:
output_folders = [i for i in os.listdir('.')
                  if os.path.isdir(os.path.abspath(i)) and '_output' in i]  

In [ ]:
xl_files = [os.path.join(folder, file)
             for folder in output_folders
             for file in os.listdir(folder)
             if os.path.splitext(file)[1] in ('.xlsx', '.xls')]

csv_files = [os.path.join(folder, file)
             for folder in output_folders
             for file in os.listdir(folder)
             if os.path.splitext(file)[1] == '.csv']

In [ ]:
def find_headers_row(file, sheet):
    headers_row = 0
    while True:
        with pd.ExcelFile(file) as xl:
            df = xl.parse(sheet, header=headers_row)
            width, height = df.shape
            for heading in df.columns:
                if 'ISBN' in str(heading):
                    return headers_row
            else:
                headers_row += 1
        if headers_row > height:
            break
    return None


def create_nested_dict(dictionary, key, subkey, value):
    if key not in dictionary:
        dictionary[key] = {subkey: value}
    else:
        dictionary[key][subkey] = value     
    return dictionary

In [ ]:
all_df_dict = dict()
for file in xl_files:
    with pd.ExcelFile(file) as xl:
        for sheet in xl.sheet_names:
            headers_row = find_headers_row(file, sheet)
            if isinstance(headers_row, int):
                df = xl.parse(sheet, header=headers_row)
                all_df_dict = create_nested_dict(all_df_dict, file, sheet, df)
for file in csv_files:
    df = pd.read_csv(file)
    all_df_dict[file] = {'default': df}

## errror checking

In [ ]:
print(len(all_df_dict))
for key in sorted(all_df_dict.keys()):
    print(key)

In [ ]:
# alpha_unique_headers

headers_set, headers_instances = set(), dict()

for file, sheets in all_df_dict.items():
    for sheet, df in sheets.items():
        for i in df.columns:
            if i in headers_instances:
                headers_instances[i] += 1
            else:
                headers_instances[i] = 1
        headers_set.update({i for i in df.columns})
        
alpha_unique_headers = sorted(list(headers_set))

In [ ]:
headers_instances

In [ ]:
flat_df = [df for file, sheets in all_df_dict.items() for sheet, df in sheets.items()]

In [ ]:
list_print_isbn = list()

for df in flat_df:
    for column in df.columns:
        if 'print' in column.lower() and 'isbn' in column.lower():
            for item in df[column]:
                list_print_isbn.append(item)

In [ ]:
len(list_print_isbn), len(set(list_print_isbn)), "duplicated ISBNs: {}".format(len(list_print_isbn) - len(set(list_print_isbn)))

In [ ]:
flat_df[1].info()

In [ ]:
flat_df[1]['LC Call Number']

In [ ]:
for num, df in enumerate(flat_df):
    for header in df:
        if 'LC' in header:
            print(num)

In [ ]:
set_a = set()
set_a.update([4, 5])

In [ ]:
set_a